<a href="https://colab.research.google.com/github/JiaUF/OnlineQuantumLab/blob/main/BB84_QKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Before running the codes, please install qiskit in the Colab terminal by using the commands:   
> pip install qiskit.  
> pip install 'qiskit[visualization]'.  
> pip install qiskit-aer.  

# BB84 Quantum Key Distribution Protocol

BB84 is the first and probably the most famous quantum key distribution (QKD) protocal. It allows two parties—traditionally named Alice and Bob—to generate a shared, secret cryptographic key, with the unique property that any eavesdropping attempt by a third party (Eve) can be detected.

In this notebook, we will walk through the steps of the BB84 protocol using Python and Qiskit to simulate the process and visualize how secure quantum communication is established. The key ideas involve:
- Quantum bits (qubits) in different bases (Z and X)
- Random basis selection
- Quantum state collapse after measurement


In [ ]:
# Imports
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import random
import matplotlib.pyplot as plt

### Step 1: Generate Random Bits and Bases
Alice randomly chooses a bit (0 or 1) and a basis (Z or X) for each qubit, and send to Bob. Here, for basis Z, 0 and 1 represent $|0\rangle$ and $|1\rangle$, respectively; for basis X, 0 and 1 represent $|-\rangle$ and $|+\rangle$, respectively.

Bob also randomly chooses a basis to measure each qubit that he receives from Alice.

In [ ]:
# Number of qubits
n = 50

# Alice's bits and bases
alice_bits = [random.randint(0, 1) for _ in range(n)]
alice_bases = [random.choice(['Z', 'X']) for _ in range(n)]

# Bob's bases
bob_bases = [random.choice(['Z', 'X']) for _ in range(n)]

### Step 2: Simulate Qubit Transmission and Measurement
For each qubit:
- Alice prepares the qubit in the correct quantum state.
- Bob measures it in his chosen basis.

In [ ]:
# Load aer_simulator as the backend. Here we run on aer quantum simulator.
backend = Aer.get_backend('aer_simulator')
print(backend)

bob_results = []

for bit, a_basis, b_basis in zip(alice_bits, alice_bases, bob_bases):

    # create a quantum circuit with 1 quantum bit (initialized in the state ∣0⟩)
    # and 1 classical bit
    qc = QuantumCircuit(1, 1)

    # Prepare Alice's qubit
    if bit == 1:
        # apply X gate to qubit 0, the numbering starts with 0
        qc.x(0)
    if a_basis == 'X':
        qc.h(0)

    # Apply Bob's basis choice
    if b_basis == 'X':
        qc.h(0)

    # qc.measure makes the measurement of qubit 0 to a classical bit 0.
    # The numbering starts with 0
    qc.measure(0, 0)

    # Transpile and run
    new_qc = transpile(qc, backend)

    # You can print the initial quantum circuit qc and the new quantum circuit
    # new_qc to visualize how the quantum compiler simplifies the circuit
    #print(qc)
    #print(new_qc)

    # shots = 1 means Run the circuit just once (i.e., generate one sample
    # measurement).
    job = backend.run(new_qc, shots=1, memory=True)
    result = job.result()
    measured = int(result.get_memory()[0])
    bob_results.append(measured)

    print(f" Alice's basis: {a_basis}. Alice's bit: {bit}. Bob's basis: {b_basis}. Bob measured: {measured}")
    print()

print("Simulation complete.")
print("Bob's results:", bob_results)

AerSimulator('aer_simulator')
 Alice's basis: X. Alice's bit: 1. Bob's basis: X. Bob measured: 1

 Alice's basis: Z. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: X. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: X. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: X. Alice's bit: 1. Bob's basis: X. Bob measured: 1

 Alice's basis: Z. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: Z. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: Z. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: X. Alice's bit: 1. Bob's basis: Z. Bob measured: 0

 Alice's basis: Z. Alice's bit: 0. Bob's basis: X. Bob measured: 0

 Alice's basis: X. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: X. Alice's bit: 1. Bob's basis: Z. Bob measured: 1

 Alice's basis: Z. Alice's bit: 0. Bob's basis: X. Bob measured: 1

 Alice's basis: X. Alice's bit: 0. Bob's basis: Z. Bob measured: 0

 Alice's basis: Z.

### Step 3: Sift Key
Alice and Bob compare bases over a classical channel and keep only the bits where their bases match.

In [ ]:
from typing import Counter
# Key sifting
states_same_basis = []
states_different_basis = []
for a_bit, a_basis, b_basis, b_bit in zip(alice_bits, alice_bases, bob_bases, bob_results):
    if a_basis == b_basis:
        states_same_basis.append((a_bit, b_bit))
    else:
        states_different_basis.append((a_bit, b_bit))


print("Sifted key (Alice, Bob):")
print(states_same_basis)
print(Counter(states_same_basis))
print()
print("The states (Alice, Bob) when Alice and Bob chose different basis:")
print(states_different_basis)
print(Counter(states_different_basis))

Sifted key (Alice, Bob):
[(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (0, 0), (1, 1), (0, 0), (1, 1), (0, 0), (0, 0), (0, 0), (0, 0), (1, 1), (1, 1), (0, 0), (1, 1), (1, 1), (0, 0), (0, 0), (1, 1), (1, 1), (1, 1), (0, 0), (1, 1), (0, 0), (0, 0), (1, 1)]
Counter({(1, 1): 17, (0, 0): 12})

The states (Alice, Bob) when Alice and Bob chose different basis:
[(1, 1), (1, 1), (1, 0), (0, 0), (1, 1), (1, 1), (0, 1), (0, 0), (1, 1), (1, 1), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, 0), (0, 1), (1, 0)]
Counter({(1, 0): 8, (1, 1): 6, (0, 1): 4, (0, 0): 3})
